In [1]:
import json
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import yaml
from box import ConfigBox
from sklearn.metrics import f1_score
from tqdm import tqdm


### Set web serice URL

In [2]:
# local testing
API_URL = "http://0.0.0.0:8080" 
# testing deployed service
# API_URL = "https://<YOUR_APP_NAME>.fly.dev" 

API_URL

'http://0.0.0.0:8080'

### Load feature columns and data

In [3]:
def load_params(params_path):
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
        params = ConfigBox(params)
    return params


proj_path = Path(os.getcwd()).parent.absolute()
params = load_params(proj_path/'params.yaml')
feat_cols = params.base.feat_cols
targ_col = params.base.targ_col
feat_cols, targ_col

(BoxList(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']),
 'Exited')

In [4]:
df = pd.concat([pd.read_csv(proj_path/'data'/'raw'/f'Churn_Modelling_{country}.csv') for country in ['France', 'Spain']])
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
2,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
3,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
4,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0


### Test `/predict` endpoint by sending one sample request

In [5]:
my_obj = {
    "data": [
            {
      "CreditScore": 619,
      "Age": 42,
      "Tenure": 2,
      "Balance": 0,
      "NumOfProducts": 1,
      "HasCrCard": 1,
      "IsActiveMember": 1,
      "EstimatedSalary": 101348.88
            }
        ]
    }
x = requests.post(API_URL + '/predict', json = my_obj)
if x.ok:
    probs = x.json()
else:
    x.raise_for_status()

probs

[0.6985236842105264]

### Create a functions for calling `/predict` endpoint

In [6]:
def get_prob(obj):
    x = requests.post(API_URL + '/predict', json = obj)
    if x.ok:
        probs = x.json()
    else:
        x.raise_for_status()
    return probs

get_prob(my_obj)

[0.6985236842105264]

### Check what would f1-scores look like if we send the data from the same geographies that the model was trained on 

In [7]:
import time


def send_sample_requests(df):
    f1_score_list = []
    for _ in tqdm(range(50)):
        df_sample = df.sample(n=60)
        y_true = df_sample[targ_col].values
        obj = {"data": df_sample[feat_cols].to_dict('records')}
        probs = get_prob(obj)
        y_pred = np.array([prob < 0.5 for prob in probs]).astype(int)
        f1 = f1_score(y_true, y_pred)
        f1_score_list.append(f1)
        time.sleep(1)
    return f1_score_list
        
f1_score_list = send_sample_requests(df)
f1_score_list

100%|██████████| 50/50 [00:50<00:00,  1.02s/it]


[0.7777777777777778,
 0.8,
 0.7999999999999999,
 0.9285714285714286,
 0.8571428571428571,
 0.8333333333333334,
 0.8571428571428571,
 0.9166666666666666,
 0.9411764705882353,
 1.0,
 0.9411764705882353,
 0.888888888888889,
 0.8421052631578948,
 1.0,
 0.9411764705882353,
 0.8888888888888888,
 0.8571428571428572,
 0.8333333333333333,
 0.9333333333333333,
 1.0,
 0.8,
 0.9333333333333333,
 0.923076923076923,
 0.888888888888889,
 0.823529411764706,
 1.0,
 0.823529411764706,
 1.0,
 0.8750000000000001,
 0.9473684210526316,
 0.8000000000000002,
 0.8333333333333333,
 0.9090909090909091,
 1.0,
 1.0,
 1.0,
 0.923076923076923,
 0.6153846153846154,
 0.8484848484848484,
 0.9285714285714286,
 0.8571428571428571,
 1.0,
 0.9523809523809523,
 0.8333333333333333,
 0.9,
 0.8,
 1.0,
 1.0,
 0.8235294117647058,
 1.0]

### What would f1-scores look like for input data from new geography?

In [8]:
df_germany =  pd.read_csv(proj_path/'data'/'more_data'/'Churn_Modelling_Germany.csv')

f1_score_list = send_sample_requests(df_germany)
f1_score_list


100%|██████████| 50/50 [00:50<00:00,  1.02s/it]


[0.6451612903225806,
 0.6060606060606061,
 0.3870967741935484,
 0.48,
 0.5333333333333333,
 0.5,
 0.4210526315789474,
 0.4761904761904762,
 0.6666666666666667,
 0.46153846153846156,
 0.35294117647058826,
 0.2857142857142857,
 0.5161290322580645,
 0.3076923076923077,
 0.4666666666666667,
 0.5,
 0.5217391304347825,
 0.5806451612903225,
 0.4210526315789473,
 0.48275862068965514,
 0.4666666666666666,
 0.5625,
 0.39999999999999997,
 0.5,
 0.3076923076923077,
 0.3809523809523809,
 0.3125,
 0.2222222222222222,
 0.6666666666666666,
 0.5454545454545454,
 0.4137931034482759,
 0.4166666666666667,
 0.3448275862068966,
 0.4444444444444445,
 0.5217391304347826,
 0.39999999999999997,
 0.3448275862068965,
 0.5,
 0.38095238095238093,
 0.3846153846153846,
 0.2222222222222222,
 0.4848484848484849,
 0.5,
 0.5882352941176471,
 0.4,
 0.5806451612903226,
 0.4210526315789473,
 0.3333333333333333,
 0.44444444444444436,
 0.3333333333333333]

### ... as expected, the values are much lower.
### But, typically, we can't compute model metrics on production data right away 
### because ground truth labels might not be available until much-much later


### All we can do is try and look at "proxy" metrics that measure statistical differences (distances) between train data and production data

In [9]:
x = requests.get(API_URL + '/drift_data')
if x.ok:
    data = x.json()
else:
    x.raise_for_status()

df_p_vals = pd.DataFrame(json.loads(data))
df_p_vals

,time,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,2024-09-24 13:50:29.818916,0.764640,0.883920,0.993834,8.030479e-01,0.998382,0.999937,0.999982,0.783409
1,2024-09-24 13:50:30.800679,0.914495,0.879815,0.494441,9.553274e-01,1.000000,0.953637,1.000000,0.997535
2,2024-09-24 13:50:31.819915,0.569845,0.982851,0.985880,3.957894e-01,0.945641,1.000000,0.901523,0.933524
3,2024-09-24 13:50:32.839429,0.697242,0.026074,0.750795,7.871491e-01,0.925950,0.800531,0.999087,0.056871
4,2024-09-24 13:50:33.860384,0.835000,0.873823,0.970229,6.695728e-02,0.999997,1.000000,1.000000,0.175025
...,...,...,...,...,...,...,...,...,...
95,2024-09-24 13:52:35.660063,0.343482,0.176391,0.899021,0.000000e+00,0.750037,1.000000,0.990236,0.827127
96,2024-09-24 13:52:36.675391,0.784556,0.810370,0.494441,0.000000e+00,0.534482,0.993182,0.405653,0.829540
97,2024-09-24 13:52:37.693413,0.622219,0.000873,0.802986,8.450530e-16,0.995000,0.999993,1.000000,0.165921
98,2024-09-24 13:52:38.714845,0.151717,0.769978,0.302424,0.000000e+00,0.206561,0.733423,0.317147,0.323171


### Values below the threshold (e.g. 0.05) indicate data drift

In [10]:
import plotly_express as px

fig = px.line(df_p_vals, x='time', y=feat_cols)
fig.add_hline(y=0.05, line_color='red')